In [14]:
import pandas as pd
import numpy as np

# Tổng quan

In [15]:
df = pd.read_csv('filled.csv')
df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,0.0,5.0,-1.00,0.0,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,0.0,4.0,-1.00,0.0,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,0.0,5.0,-1.00,0.0,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,0.0,1.0,-1.00,0.0,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [ ]:
df.columns

In [17]:
df = df.drop(['id', 'Name', 'City'], axis=1)
df.columns

Index(['Gender', 'Age', 'Working Professional or Student', 'Profession',
       'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction',
       'Job Satisfaction', 'Sleep Duration', 'Dietary Habits', 'Degree',
       'Have you ever had suicidal thoughts ?', 'Work/Study Hours',
       'Financial Stress', 'Family History of Mental Illness', 'Depression'],
      dtype='object')

In [18]:
for column in df.columns:
    print(df[column].value_counts(), end='\n----------\n')


Gender
Male      77350
Female    63136
Name: count, dtype: int64
----------
Age
56.0    5241
49.0    5094
38.0    4557
53.0    4523
57.0    4392
        ... 
60.0    2497
26.0    2104
22.0    2063
30.0    1906
32.0    1781
Name: count, Length: 43, dtype: int64
----------
Working Professional or Student
Working Professional    112604
Student                  27882
Name: count, dtype: int64
----------
Profession
Student              27886
Teacher              24900
Other                 8572
Content Writer        7811
Architect             4362
                     ...  
City Manager             1
FamilyVirar              1
Family Consultant        1
B.Com                    1
Yuvraj                   1
Name: count, Length: 65, dtype: int64
----------
Academic Pressure
0.0    112604
3.0      7460
5.0      6293
4.0      5154
1.0      4799
2.0      4176
Name: count, dtype: int64
----------
Work Pressure
0.0    27882
2.0    24342
4.0    22476
5.0    22387
3.0    21868
1.0    21531
Name: cou

- Cần phân loại lại Age 
- Cần phân loại, xử lý các dữ liệu lỗi của cột Profession
- Cần phân loại lại các khung điểm CGPA
- Cần xử lý lại các khung giờ Sleep Duration
- Cần xử lý các bộ dữ liệu lỗi Dietary Habbits
- Cần xử lý lại các dữ liệu lỗi cột Degree
- Xử lý lại label cột work/study hours

# Xử lý Age

In [ ]:
df['Age'].value_counts()

Ta phân loại các tuổi trên thành 8 nhóm tuổi, bao gồm:
- <=10: Không xét đến ở đây
- 11-17: Giai đoạn dậy thì, thay đổi lớn về thể chất và tâm sinh lý, phát triển bản sắc cá nhân. Các vấn đề thường gặp là rối loạn lo âu, rối loạn ăn uống, trầm cảm, hành vi nguy cơ, áp lực học tập, định hướng nghề nghiệp, các mối quan hệ tình cảm.
- 18-24: Nhóm này bao gồm những người trẻ mới trưởng thành, đang trong giai đoạn chuyển tiếp quan trọng của cuộc đời, đối mặt với nhiều áp lực học tập, công việc và các mối quan hệ.
- 25-34: Nhóm này đang trong giai đoạn xây dựng sự nghiệp và gia đình, có thể gặp căng thẳng từ công việc, tài chính và trách nhiệm gia đình.
- 35-44: Nhóm này có thể đang ở đỉnh cao sự nghiệp, đồng thời phải cân bằng giữa công việc và gia đình, có thể trải qua "khủng hoảng tuổi trung niên".
- 45-54: Nhóm này có thể phải đối mặt với các vấn đề sức khỏe, thay đổi vai trò trong gia đình (con cái trưởng thành), và chuẩn bị cho giai đoạn nghỉ hưu.
- 55-60: Nhóm này đang trong giai đoạn chuẩn bị nghỉ hưu, có thể gặp các vấn đề về sức khỏe mãn tính, thay đổi vai trò xã hội.
- \>=61: Không xét đến ở đây

In [ ]:
# Các điều kiện
conditions = [
    df['Age'] <= 10,
    (df['Age'] >= 11) & (df['Age'] <= 17),
    (df['Age'] >= 18) & (df['Age'] <= 24),
    (df['Age'] >= 25) & (df['Age'] <= 34),
    (df['Age'] >= 35) & (df['Age'] <= 44),
    (df['Age'] >= 45) & (df['Age'] <= 54),
    (df['Age'] >= 55) & (df['Age'] <= 60),
    df['Age'] >= 61
]

# Các giá trị tương ứng
labels = ['<=10', '11-17', '18-24', '25-34', '35-44', '45-54', '55-60', '>=61']

# Áp dụng phân loại
df['AgeGroup'] = np.select(conditions, labels, default='Unknown')

In [ ]:
df['AgeGroup'].value_counts()

In [ ]:
df = df.drop(['Age'], axis=1)

In [ ]:
df.columns

# Xử lý Profession

In [ ]:
pd.options.display.max_rows = None
print(df['Profession'].value_counts())
pd.options.display.max_rows = 10

*   **Giáo dục (Education):**
    *   `Teacher`
    *   `Educational Consultant`
    *   `Academic`
    *   `M.Ed`
    *   `B.Ed`

*   **Y tế (Healthcare):**
    *   `Doctor`
    *   `Pharmacist`
    *   `Medical Doctor`
    *   `MBBS`

*   **Kinh doanh & Quản lý (Business & Management):**
    *   `Business Analyst`
    *   `Consultant`
    *   `Entrepreneur`
    *   `HR Manager`
    *   `Marketing Manager`
    *   `Manager`
    *   `Family Consultant`
    *   `City Manager`

*   **Kỹ thuật & Công nghệ (Engineering & Technology):**
    *   `Software Engineer`
    *   `Data Scientist`
    *   `Mechanical Engineer`
    *   `Civil Engineer`
    *   `Architect`
    *   `UX/UI Designer`
    *   `BE`

*   **Tài chính & Kế toán (Finance & Accounting):**
    *   `Financial Analyst`
    *   `Accountant`
    *   `Finanancial Analyst`
    *   `Investment Banker`

*   **Khoa học & Nghiên cứu (Science & Research):**
    *   `Chemist`
    *   `Researcher`
    *   `Research Analyst`
    *   `PhD`

*   **Luật (Law):**
    *   `Lawyer`
    *   `Judge`
    *   `LLM`

*   **Nghệ thuật & Thiết kế (Art & Design):**
    *   `Graphic Designer`

*   **Dịch vụ (Services):**
    *   `Customer Support`
    *   `Travel Consultant`
    *   `Plumber`
    *   `Chef`
    *   `Electrician`
    *   `Sales Executive`
    *   `Pilot`
    *   `Content Writer`
     *   `Digital Marketer`
    * `Analyst`

*   **Sinh viên (Student):**
    *   `Student`
    *   `BCA`
    *   `MBA`
    *   `BBA`
    *   `B.Com`

*   **Thất nghiệp (Unemployed):**
    *   `Unemployed`

*   **Khác (Other):**
    *   `Other`
    *   `Yogesh`
    *   `Unveil`
    *   `Visakhapatnam`
    *   `Pranav`
    *   `Moderate`
    *   `Nagpur`
    *   `Patna`
    *   `Dev`
    *   `Working Professional`
    *   `FamilyVirar`
    *   `Yuvraj`
    * `Profession`

In [ ]:


# Các điều kiện cho từng nhóm nghề nghiệp
conditions = [
    df['Profession'].isin(['Teacher', 'Educational Consultant', 'Academic', 'M.Ed', 'B.Ed']),  # Giáo dục
    df['Profession'].isin(['Doctor', 'Pharmacist', 'Medical Doctor', 'MBBS']),  # Y tế
    df['Profession'].isin(['Business Analyst', 'Consultant', 'Entrepreneur', 'HR Manager', 'Marketing Manager', 'Manager', 'Family Consultant', 'City Manager']),  # Kinh doanh & Quản lý
    df['Profession'].isin(['Software Engineer', 'Data Scientist', 'Mechanical Engineer', 'Civil Engineer', 'Architect', 'UX/UI Designer', 'BE']),  # Kỹ thuật & Công nghệ
    df['Profession'].isin(['Financial Analyst', 'Accountant', 'Finanancial Analyst', 'Investment Banker']),  # Tài chính & Kế toán
    df['Profession'].isin(['Chemist', 'Researcher', 'Research Analyst', 'PhD']),  # Khoa học & Nghiên cứu
    df['Profession'].isin(['Lawyer', 'Judge', 'LLM']),  # Luật
    df['Profession'].isin(['Graphic Designer']),  # Nghệ thuật & Thiết kế
    df['Profession'].isin(['Customer Support', 'Travel Consultant', 'Plumber', 'Chef', 'Electrician', 'Sales Executive', 'Pilot', 'Content Writer', 'Digital Marketer', 'Analyst']),  # Dịch vụ
    df['Profession'].isin(['Student', 'BCA', 'MBA', 'BBA', 'B.Com']),  # Sinh viên
]

# Các nhãn tương ứng với từng nhóm nghề nghiệp
labels = [
    'Education',
    'Healthcare',
    'Business & Management',
    'Engineering & Technology',
    'Finance & Accounting',
    'Science & Research',
    'Law',
    'Art & Design',
    'Services',
    'Student',
]

# Áp dụng phân loại và gán các giá trị không khớp với bất kỳ điều kiện nào thành 'Other'
df['Profession_Group'] = np.select(conditions, labels, default='Other')

# In ra kết quả
print(df['Profession_Group'].value_counts()) 

# Xử lý CGPA

In [ ]:
pd.options.display.max_rows = None
print(df['CGPA'].value_counts())
pd.options.display.max_rows = 10